In [1]:
import os
import glob
import tqdm
import numpy as np
import scipy.ndimage
import scipy.misc
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from sklearn.cluster import KMeans
import subprocess
import shutil
import cv2
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


In [2]:
import azureml
from azureml.core import Workspace, Run

In [3]:
# import os
# os.system("pip install opencv-python")

In [4]:
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.17


In [5]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

Found the config file in: /home/nbuser/library/config.json
AvinashMLWorkspace	eastus2	serving-preprod	eastus2


In [6]:
experiment_name = 'recommendation'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 1)

In [8]:
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")

In [9]:
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. cpucluster


In [10]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name, ds.account_key)

AzureFile avinashmlworks6976879219 azureml-filestore-cb8f635d-7278-4d10-8a83-8559c11cd70e 3JX3qNUAJR0AKF5cpcUYP21fMOvPNvMnhJl2IZ/xbAsYcXZmGThF2oGx63ZcMB8+BYVPJso/+bH0byt/u/j7LQ==


In [11]:
ds.upload(src_dir='DataSet', target_path='data', overwrite=True, show_progress=True)

Uploading DataSet/103.262.77.jpg
Uploading DataSet/400.441.96.jpg
Uploading DataSet/590.483.97.jpg
Uploading DataSet/303.270.68.jpg
Uploading DataSet/102.604.60.jpg
Uploading DataSet/390.108.90.jpg
Uploading DataSet/801.638.04.jpg
Uploading DataSet/991.278.11.jpg
Uploading DataSet/802.017.40.jpg
Uploading DataSet/502.400.31.jpg
Uploading DataSet/302.625.28.jpg
Uploading DataSet/402.808.43.jpg
Uploading DataSet/791.278.07.jpg
Uploading DataSet/900.954.28.jpg
Uploading DataSet/602.470.51.jpg
Uploading DataSet/202.396.99.jpg
Uploading DataSet/002.460.78.jpg
Uploading DataSet/501.784.11.jpg
Uploading DataSet/902.639.02.jpg
Uploading DataSet/902.396.67.jpg
Uploading DataSet/103.203.41.jpg
Uploading DataSet/490.912.49.jpg
Uploading DataSet/498.984.35.jpg
Uploading DataSet/891.278.02.jpg
Uploading DataSet/503.056.16.jpg
Uploading DataSet/102.335.32.jpg
Uploading DataSet/101.168.49.jpg
Uploading DataSet/902.675.56.jpg
Uploading DataSet/490.473.41.jpg
Uploading DataSet/690.473.40.jpg
Uploading 

Uploaded DataSet/2/391.278.09.jpg, 67 files out of an estimated total of 212
Uploading DataSet/3/590.335.55.jpg
Uploaded DataSet/2/191.278.10.jpg, 68 files out of an estimated total of 212
Uploading DataSet/3/690.335.45.jpg
Uploaded DataSet/2/502.595.01.jpg, 69 files out of an estimated total of 212
Uploading DataSet/3/991.506.46.jpg
Uploaded DataSet/2/591.278.08.jpg, 70 files out of an estimated total of 212
Uploading DataSet/302.249.04.jpg
Uploaded DataSet/2/791.278.07.jpg, 71 files out of an estimated total of 212
Uploading DataSet/302.332.44.jpg
Uploaded DataSet/201.150.38.jpg, 72 files out of an estimated total of 212
Uploading DataSet/302.882.36.jpg
Uploaded DataSet/1/391.505.88.jpg, 73 files out of an estimated total of 212
Uploading DataSet/391.278.09.jpg
Uploaded DataSet/202.085.27.jpg, 74 files out of an estimated total of 212
Uploading DataSet/391.505.88.jpg
Uploaded DataSet/1/102.335.32.jpg, 75 files out of an estimated total of 212
Uploading DataSet/399.030.41.jpg
Uploaded

Uploaded DataSet/591.278.08.jpg, 151 files out of an estimated total of 212
Uploading DataSet/802.340.19.jpg
Uploaded DataSet/5/790.462.60.jpg, 152 files out of an estimated total of 212
Uploading DataSet/802.396.82.jpg
Uploaded DataSet/502.675.58.jpg, 153 files out of an estimated total of 212
Uploading DataSet/802.671.37.jpg
Uploaded DataSet/502.595.01.jpg, 154 files out of an estimated total of 212
Uploading DataSet/890.066.83.jpg
Uploaded DataSet/7/399.030.41.jpg, 155 files out of an estimated total of 212
Uploading DataSet/890.417.09.jpg
Uploaded DataSet/7/598.501.07.jpg, 156 files out of an estimated total of 212
Uploading DataSet/9/002.224.40.jpg
Uploaded DataSet/7/202.396.80.jpg, 157 files out of an estimated total of 212
Uploading DataSet/9/100.998.97.jpg
Uploaded DataSet/7/902.396.67.jpg, 158 files out of an estimated total of 212
Uploading DataSet/9/103.203.41.jpg
Uploaded DataSet/701.032.50.jpg, 159 files out of an estimated total of 212
Uploading DataSet/9/302.625.28.jpg
U

$AZUREML_DATAREFERENCE_a7ba35e3f89a4689b0aca0a45c0c8661

In [12]:
my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

In [13]:
images_name  = []
casting='unsafe'
def extract_vector(path):
    resnet_feature_list = []

    for im in glob.glob(path):
        print(im.split("/")[1])
        images_name.append(im.split("/")[1])
        im = cv2.imread(im)
        im = cv2.resize(im,(224,224))
        img = preprocess_input(np.expand_dims(im.copy(), axis=0).astype(float))
        resnet_feature = my_new_model.predict(img)
        resnet_feature_np = np.array(resnet_feature)
        resnet_feature_list.append(resnet_feature_np.flatten())

    return np.array(resnet_feature_list)

In [14]:
#object_type = "chair"
#path = "/Users/sumit.khaitan/Desktop/livspace/"+object_type+"/"
path = "DataSet/"
kmeans = KMeans(n_clusters=10, random_state=0).fit(extract_vector(path+"*.jpg"))
print(kmeans.labels_)

for j in range(0,10):
    if not os.path.exists(path+str(j)):
        os.makedirs(path+str(j))
    else:
        shutil.rmtree(path+str(j))
        os.makedirs(path+str(j))


for j in range(0,len(kmeans.labels_)):
    filename = path+images_name[j]
    dest_dir = path+str(kmeans.labels_[j])
    print(filename+" "+dest_dir)
    shutil.copy(filename, dest_dir)

103.262.77.jpg
400.441.96.jpg
590.483.97.jpg
303.270.68.jpg
102.604.60.jpg
390.108.90.jpg
801.638.04.jpg
991.278.11.jpg
802.017.40.jpg
502.400.31.jpg
302.625.28.jpg
402.808.43.jpg
791.278.07.jpg
900.954.28.jpg
602.470.51.jpg
202.396.99.jpg
002.460.78.jpg
501.784.11.jpg
902.639.02.jpg
902.396.67.jpg
103.203.41.jpg
490.912.49.jpg
498.984.35.jpg
891.278.02.jpg
503.056.16.jpg
102.335.32.jpg
101.168.49.jpg
902.675.56.jpg
490.473.41.jpg
690.473.40.jpg
690.335.45.jpg
402.805.17.jpg
802.251.66.jpg
500.583.76.jpg
901.011.13.jpg
090.066.63.jpg
702.790.70.jpg
790.066.93.jpg
601.963.58.jpg
101.150.67.jpg
602.460.80.jpg
598.501.07.jpg
591.278.08.jpg
702.381.88.jpg
902.642.18.jpg
301.165.51.jpg
202.800.33.jpg
902.874.13.jpg
402.177.95.jpg
702.068.04.jpg
290.066.95.jpg
502.954.72.jpg
701.032.50.jpg
003.015.26.jpg
401.536.99.jpg
903.200.97.jpg
103.064.39.jpg
002.224.40.jpg
802.671.37.jpg
201.150.38.jpg
402.288.12.jpg
001.652.13.jpg
802.340.19.jpg
802.396.82.jpg
202.396.80.jpg
202.013.33.jpg
902.530.74

In [15]:
import os
script_folder = 'recommendation_script'
os.makedirs(script_folder, exist_ok=True)

In [16]:
%%writefile $script_folder/train.py

import argparse
import os
import glob
import tqdm
import numpy as np
import scipy.ndimage
import scipy.misc
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from sklearn.cluster import KMeans
import subprocess
import shutil
import cv2
from keras.applications.imagenet_utils import preprocess_input


from sklearn.externals import joblib

from azureml.core import Run

images_name  = []
def extract_vector(path):
    resnet_feature_list = []

    for im in glob.glob(path):
        print(im.split("/")[1])
        images_name.append(im.split("/")[1])
        im = cv2.imread(im)
        im = cv2.resize(im,(224,224))
        img = preprocess_input(np.expand_dims(im.copy(), axis=0).astype(float))
        resnet_feature = my_new_model.predict(img)
        resnet_feature_np = np.array(resnet_feature)
        resnet_feature_list.append(resnet_feature_np.flatten())

    return np.array(resnet_feature_list)

# let user feed in 2 parameters, the location of the data files (from datastore)
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()
print("sumit"+args.data_folder)
data_folder = os.path.join(args.data_folder, 'data')


print(data_folder)
cwd = os.getcwd()
print(cwd)

# get hold of the current run
run = Run.get_context()

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False
kmeans = KMeans(n_clusters=10, random_state=0).fit(extract_vector(data_folder+"/*.jpg"))
 

print('Predict the test set '+str(kmeans))
print("sumit"+args.data_folder)
print(data_folder)
print(os.listdir(data_folder))
print(kmeans.predict(extract_vector(data_folder+"/902.642.18.jpg")))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=kmeans, filename='outputs/kmeans.pkl')
joblib.dump(value=my_new_model, filename='outputs/new_model.pkl')

Overwriting recommendation_script/train.py


In [17]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.as_mount(),
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['scikit-learn','keras','tqdm','numpy','opencv','scipy'])

print(ds.as_mount())

$AZUREML_DATAREFERENCE_workspacefilestore


In [18]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
recommendation,recommendation_1552778182_b6bc5950,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [19]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [20]:
run.wait_for_completion(show_output=False) # specify True for a verbose log

{'runId': 'recommendation_1552778182_b6bc5950',
 'target': 'cpucluster',
 'status': 'Finalizing',
 'startTimeUtc': '2019-03-16T23:16:41.842171Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '49a5ab63-641a-4544-84be-71407a3368f4'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': ['--data-folder', '$AZUREML_DATAREFERENCE_workspacefilestore'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'cpucluster',
  'DataReferences': {'workspacefilestore': {'DataStoreName': 'workspacefilestore',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
      {'pip': ['azureml-defaults']},


In [21]:
print(run.get_metrics())

{}


In [22]:
print(run.get_file_names())

['azureml-logs/60_control_log.txt', 'azureml-logs/80_driver_log.txt', 'outputs/kmeans.pkl', 'outputs/new_model.pkl']


In [23]:
# register model 
model = run.register_model(model_name='recommendation', model_path='outputs/kmeans.pkl')
print(model.name, model.id, model.version, sep = '\t')


model_new = run.register_model(model_name='new_model', model_path='outputs/new_model.pkl')
print(model_new.name, model_new.id, model_new.version, sep = '\t')

recommendation	recommendation:8	8
new_model	new_model:4	4
